# Machine Learning with scikit-learn

When you have explored a series of steps that is useful for your task—or for a related family of tasks—you would like to package those in a less ad hoc and more reusable way.  Certainly wrapping a set of steps in a simple factory function is not difficult.  But for most flexibility it is best to take advantage of scikit-learn **pipelines**.

In [1]:
# Some libraries tend to be in flux for their dependency versions
import warnings
warnings.simplefilter("ignore")

## Imperative Sequential Processing

Let us repeat a similar construction of building and training a model as we have seen previously.  Here we carry out the steps imperatively, in the sequence we worked out in previous lessons.

In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### Load the data

First load the data; this step will not become part of the pipeline since loading the initial data may occur in various ways that are use dependent.  We will use a cross validation score rather than do a train/test split up front.  In the next lesson we will see how this can be more robust than a simple train/test split.

In [3]:
from sklearn.datasets import load_breast_cancer
cancer = load_breast_cancer()

# From here on, we refer to features and target by the
# generic X, y rather than tie it to the dataset
X, y = cancer.data, cancer.target
X.shape

(569, 30)

### Generate Synthetic Features

We think the model may perform better with polynomial features that get at the interactions of multiple features.

In [4]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(2)
X_poly = poly.fit_transform(X)
X_poly.shape

(569, 496)

### Scale the Data

Scale the data for better performance in subsequent models.

In [5]:
from sklearn.preprocessing import MinMaxScaler

# compute minimum and maximum on the training data
scaler = MinMaxScaler()
scaler.fit(X_poly)
# rescale training data
X_poly_scaled = scaler.transform(X_poly)
X_poly_scaled.shape

(569, 496)

### Select Most Important Engineered Features

Since we have increased the number of features to an unweildy number, let us select only the top most important few.

In [6]:
from sklearn.feature_selection import SelectPercentile

select = SelectPercentile(percentile=20)
select.fit(X_poly_scaled, y)
X_selected = select.transform(X_poly_scaled)
X_selected.shape

(569, 99)

### Test Feature Engineered Data Against Model

Having gone through those steps, we would like to see how our engineered dataset performs on a model that showed some success in earlier lessons.

In [7]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(max_depth=7, random_state=1)

In [9]:
from sklearn.metrics import f1_score, make_scorer
scorer = make_scorer(f1_score)

from sklearn.model_selection import KFold
kf = KFold(5, random_state=0)

In [10]:
from sklearn.model_selection import cross_val_score
cv_scores = cross_val_score(rfc, X_selected, y, scoring=scorer, cv=kf)
print(" CV scores:", cv_scores)
print("Mean score:", np.mean(cv_scores))

 CV scores: [0.92929293 0.98461538 0.98013245 0.99408284 0.98245614]
Mean score: 0.9741159489654005


As a sanity check, let us see how we would have performed on the raw data.  This gets a moderate but significant improvement over the raw data in F1 score.

In [11]:
cv_scores = cross_val_score(rfc, X, y, scoring=scorer, cv=kf)
print("Raw data CV scores:", cv_scores)
print("    Mean raw score:", np.mean(cv_scores))

Raw data CV scores: [0.89795918 0.94573643 0.98666667 0.9704142  0.98245614]
    Mean raw score: 0.9566465251965944


## Using Pipelines

A pipeline is simply an abstraction in scikit-learn to bundle together steps like those used above into a single model interface, following the same APIs as a model itself.  A particular pipeline is likely to be somewhat domain specific in that you may learn that those particular steps are useful for e.g. cancer data, but not as useful for data with very different characteristics.

<img src="img/pipeline-diagram.png" alt="Pipeline Illustration" width="60%"/>

Image credit (CC-BY-NA): [Karl Rosaen](http://karlrosaen.com/ml/learning-log/2016-06-20/)

### Putting it together

We can easily construct a pipeline consisting of just those steps (instances that follow the scorer interface) we want.  When we instantiate the various classes, we are free to pass in parameters we know we will want; these likely reflect our previous exploration of the particular domain and its datasets.

In [12]:
from sklearn.pipeline import Pipeline

pipe = Pipeline([
    ("Polynomial features", PolynomialFeatures(2)),
    ("MinMax scaling", MinMaxScaler()),
    ("Top 20% features", SelectPercentile(percentile=20)),
    ("Random Forest", RandomForestClassifier(max_depth=7)),
])

Using the pipeline object is just like using a plain model, but all the preparation steps are bundled into a single object.

In [13]:
cv_scores = cross_val_score(pipe, 
                            X, y, 
                            scoring=make_scorer(f1_score), 
                            cv=KFold(5))

print(" Pipeline CV scores:", cv_scores)
print("Pipeline mean score:", np.mean(cv_scores))

 Pipeline CV scores: [0.91836735 0.98461538 0.98666667 0.98224852 0.98245614]
Pipeline mean score: 0.9708708118563525


We can recover (and even modify in-place) the steps of a pipeline

In [14]:
pipe.steps

[('Polynomial features', PolynomialFeatures()),
 ('MinMax scaling', MinMaxScaler()),
 ('Top 20% features', SelectPercentile(percentile=20)),
 ('Random Forest', RandomForestClassifier(max_depth=7))]

In [17]:
# We can serialize the pipeline for later use
from pickle import dump, load
dump(pipe, open('data/cancer-pipeline.pkl','wb'))

In [18]:
newpipe = load(open('data/cancer-pipeline.pkl','rb'))
cv_scores = cross_val_score(newpipe, 
                            X, y, 
                            scoring=make_scorer(f1_score), 
                            cv=KFold(5))

print(" Pipeline CV scores:", cv_scores)
print("Pipeline mean score:", np.mean(cv_scores))

 Pipeline CV scores: [0.93877551 0.98461538 0.98666667 0.97619048 0.98245614]
Pipeline mean score: 0.9737408356054973


In [19]:
pipe.fit(X, y)

Pipeline(steps=[('Polynomial features', PolynomialFeatures()),
                ('MinMax scaling', MinMaxScaler()),
                ('Top 20% features', SelectPercentile(percentile=20)),
                ('Random Forest', RandomForestClassifier(max_depth=7))])

In [20]:
pipe.predict(X)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0,
       1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1,
       1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0,

#### A pipeline factory

There is a convenience function to create pipelines.  The only difference of interest with the class constructor is that names of steps are generated for you rather than being explicitly spelled when you create a pipeline.  This is slightly more convenient but also takes away your option of giving more descriptive names for steps.

In [21]:
from sklearn.pipeline import make_pipeline
pipe = make_pipeline(
    PolynomialFeatures(2),
    MinMaxScaler(),
    SelectPercentile(percentile=20),
    RandomForestClassifier(max_depth=7))
pipe.steps

[('polynomialfeatures', PolynomialFeatures()),
 ('minmaxscaler', MinMaxScaler()),
 ('selectpercentile', SelectPercentile(percentile=20)),
 ('randomforestclassifier', RandomForestClassifier(max_depth=7))]

## Pipelines with Grid Search

A very nice feature of using a pipeline is that it "plays well" with a grid search.  In fact, you are not restrained to searching over the hyperparameters of the model step, but can also search over arguments to other steps in the pipeline.  For this, spelling is a little easier if we use the generated step names from `make_pipeline()`.

In [22]:
%%time
# Takes about a minute for this grid search
from sklearn.model_selection import GridSearchCV

params = {'polynomialfeatures__degree': [1, 2, 3],
          'selectpercentile__percentile': [10, 15, 20, 50],
          'randomforestclassifier__max_depth': [5, 7, 9],
          'randomforestclassifier__criterion': ['entropy', 'gini']}

grid = GridSearchCV(pipe, param_grid=params, cv=5)
grid.fit(X, y)

print("best cross-validation accuracy:", grid.best_score_)
print("best dataset score: ", grid.score(X, y))   # Overfitting against entire dataset
print("best parameters: ", grid.best_params_)

best cross-validation accuracy: 0.9789318428815401
best dataset score:  1.0
best parameters:  {'polynomialfeatures__degree': 3, 'randomforestclassifier__criterion': 'gini', 'randomforestclassifier__max_depth': 7, 'selectpercentile__percentile': 50}
CPU times: user 1min 56s, sys: 1.71 s, total: 1min 57s
Wall time: 1min 58s


In [23]:
model = grid.best_estimator_
cv_scores = cross_val_score(model, 
                            X, y, 
                            scoring=make_scorer(f1_score), 
                            cv=KFold(5))

print(" Grid CV scores:", cv_scores)
print("Grid mean score:", np.mean(cv_scores))

 Grid CV scores: [0.94845361 0.98461538 0.98013245 0.98809524 0.99421965]
Grid mean score: 0.9791032668936724


The model we select as `.best_estimator_` is itself a pipeline; it simply has been re-parameterized from the original pipeline, using the grid search.

In [24]:
model.steps

[('polynomialfeatures', PolynomialFeatures(degree=3)),
 ('minmaxscaler', MinMaxScaler()),
 ('selectpercentile', SelectPercentile(percentile=50)),
 ('randomforestclassifier', RandomForestClassifier(max_depth=7))]

We can examine the relative success of all the parameter combinations as well.  As we saw in a prior lesson, `.cv_results_` contains a rich collection of information beyond this also.  Although the highest degree polynomial features and highest percentage feature selection was the best estimator, in the ranking of classifiers, there is quite a bit of variation in all the parameters searched.  In particular, entirely different combinations perform only slightly worse in the example (they are close enough that it might turn out differently among the top few with different random seeds).

In [25]:
df_grid = pd.DataFrame(grid.cv_results_).set_index('rank_test_score').sort_index()
df_params = df_grid.loc[:,df_grid.columns.str.contains('param_')]
cols = [c.split('_')[-1] for c in df_params.columns]
df_params.columns = cols
df_params.head(10)

,degree,criterion,depth,percentile
rank_test_score,,,,
1,3,gini,7,50
2,3,entropy,5,50
3,3,gini,9,20
3,3,gini,9,15
3,3,gini,7,15
3,3,entropy,7,50
3,3,gini,5,15
3,3,gini,5,10
9,3,entropy,5,10


## Next lesson

**Robust train/test splits**: In this lesson we looked at the very useful pipeline interface provided by scikit-learn.  Using pipelines greatly aids in making models and processing steps reproducible and easy to distribute to colleagues (or to yourself with later projects).

The next and final lesson of this course, on robust train/test splits, will look at a variety of capabilities in scikit-learn for performing divisions between training and validation data that go beyond the basic function we used in most of these lessons.

<a href="TrainTest.ipynb"><img src="img/open-notebook.png" align="left"/></a>